<a href="https://colab.research.google.com/github/Uriel1201/HelloJava/blob/main/11_birthday/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#11. Birthday Attendance.

**Writing a query to return the fraction of students,
rounded to two decimal places, who
attended school on their birthday.**

In [1]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [3]:
data1 = {'student_id'    : [1,2,3,1,2,3,1,2,3,4],
        'school_date'    : ['03-APR-20',
                         '03-APR-20',
                         '03-APR-20',
                         '04-APR-20',
                         '04-APR-20',
                         '04-APR-20',
                         '05-APR-20',
                         '05-APR-20',
                         '05-APR-20',
                         '05-APR-20'],
         'attendance'    : [0,1,1,1,1,1,0,1,1,1]
        }

data2 = {'student_id'    : [1,2,3,4],
         'school_id'     : [2,1,1,2],
         'grade'         : [5,4,3,4],
         'date_birth'    : ['03-APR-12',
                         '04-APR-13',
                         '05-APR-14',
                         '03-APR-13']
        }
pd_attendance  = pd.DataFrame(data1)
pd_students    = pd.DataFrame(data2)
pl_attendance  = pl.LazyFrame(data1)
pl_students    = pl.LazyFrame(data2)

pd_attendance['school_date']=(pd.to_datetime(pd_attendance['school_date'],
                                             format="%d-%b-%y"
                                 )
)
pd_students['date_birth']=(pd.to_datetime(pd_students['date_birth'],
                                          format="%d-%b-%y"
                              )
)
pl_attendance=(pl_attendance.with_columns(pl.col('school_date')
                                            .str
                                            .strptime(pl.Date,
                                                      format="%d-%b-%y"
                                             )
                             )
)
pl_students=(pl_students.with_columns(pl.col('date_birth')
                                        .str
                                        .strptime(pl.Date,
                                                  format="%d-%b-%y"
                                         )
                         )
)

In [4]:
print(f'----PANDAS----\nattendance table:\n{pd_attendance}')

----PANDAS----
attendance table:
   student_id school_date  attendance
0           1  2020-04-03           0
1           2  2020-04-03           1
2           3  2020-04-03           1
3           1  2020-04-04           1
4           2  2020-04-04           1
5           3  2020-04-04           1
6           1  2020-04-05           0
7           2  2020-04-05           1
8           3  2020-04-05           1
9           4  2020-04-05           1


In [5]:
print(f'students table:\n{pd_students}')

students table:
   student_id  school_id  grade date_birth
0           1          2      5 2012-04-03
1           2          1      4 2013-04-04
2           3          1      3 2014-04-05
3           4          2      4 2013-04-03


In [6]:
pd_df1=(pd_students['student_id'].to_frame()
                                      .assign(day=pd_students['date_birth'].dt
                                                                           .day,
                                              month=pd_students['date_birth'].dt
                                                                             .month
                                       )
)
print(f'Identifying birthday only by month and day:\n{pd_df1}')

Identifying birthday only by month and day:
   student_id  day  month
0           1    3      4
1           2    4      4
2           3    5      4
3           4    3      4


In [7]:
pd_df2=(pd_attendance[['student_id','attendance']].assign(day=pd_attendance['school_date'].dt.day,
                                                          month=pd_attendance['school_date'].dt.month
                                                   )
)
print(f'identifying attendance only by month and day:\n{pd_df2}')

identifying attendance only by month and day:
   student_id  attendance  day  month
0           1           0    3      4
1           2           1    3      4
2           3           1    3      4
3           1           1    4      4
4           2           1    4      4
5           3           1    4      4
6           1           0    5      4
7           2           1    5      4
8           3           1    5      4
9           4           1    5      4


In [8]:
pd_birthday=(pd.merge(pd_df1,
                      pd_df2,
                      on=['student_id','day','month'],
                      how='left'
                )
)
fraction=round(pd_birthday['attendance'].mean()
               ,2
         )
print(f'{pd_birthday}\nfraction of birthday attendance:\n{fraction}')

   student_id  day  month  attendance
0           1    3      4         0.0
1           2    4      4         1.0
2           3    5      4         1.0
3           4    3      4         NaN
fraction of birthday attendance:
0.67


In [9]:
sample1=pl_attendance.head(5)
sample2=pl_students.head(5)
print(f'----POLARS----\nattendance table (SAMPLE):\n{sample1.collect()}')

----POLARS----
attendance table (SAMPLE):
shape: (5, 3)
┌────────────┬─────────────┬────────────┐
│ student_id ┆ school_date ┆ attendance │
│ ---        ┆ ---         ┆ ---        │
│ i64        ┆ date        ┆ i64        │
╞════════════╪═════════════╪════════════╡
│ 1          ┆ 2020-04-03  ┆ 0          │
│ 2          ┆ 2020-04-03  ┆ 1          │
│ 3          ┆ 2020-04-03  ┆ 1          │
│ 1          ┆ 2020-04-04  ┆ 1          │
│ 2          ┆ 2020-04-04  ┆ 1          │
└────────────┴─────────────┴────────────┘


In [10]:
print(f'students table (SAMPLE):\n{sample2.collect()}')

students table (SAMPLE):
shape: (4, 4)
┌────────────┬───────────┬───────┬────────────┐
│ student_id ┆ school_id ┆ grade ┆ date_birth │
│ ---        ┆ ---       ┆ ---   ┆ ---        │
│ i64        ┆ i64       ┆ i64   ┆ date       │
╞════════════╪═══════════╪═══════╪════════════╡
│ 1          ┆ 2         ┆ 5     ┆ 2012-04-03 │
│ 2          ┆ 1         ┆ 4     ┆ 2013-04-04 │
│ 3          ┆ 1         ┆ 3     ┆ 2014-04-05 │
│ 4          ┆ 2         ┆ 4     ┆ 2013-04-03 │
└────────────┴───────────┴───────┴────────────┘


In [11]:
pl_lf1=(sample1.select(pl.col('student_id')
                       ,pl.col('attendance')
                       ,pl.col('school_date')
                )
               .with_columns(birthday_day=pl.col('school_date')
                                            .dt
                                            .day(),
                             birthday_month=pl.col('school_date')
                                              .dt
                                              .month()
                  )
).collect()
print(f'Identifying attendance only by month and day:\n{pl_lf1}')

Identifying attendance only by month and day:
shape: (5, 5)
┌────────────┬────────────┬─────────────┬──────────────┬────────────────┐
│ student_id ┆ attendance ┆ school_date ┆ birthday_day ┆ birthday_month │
│ ---        ┆ ---        ┆ ---         ┆ ---          ┆ ---            │
│ i64        ┆ i64        ┆ date        ┆ i8           ┆ i8             │
╞════════════╪════════════╪═════════════╪══════════════╪════════════════╡
│ 1          ┆ 0          ┆ 2020-04-03  ┆ 3            ┆ 4              │
│ 2          ┆ 1          ┆ 2020-04-03  ┆ 3            ┆ 4              │
│ 3          ┆ 1          ┆ 2020-04-03  ┆ 3            ┆ 4              │
│ 1          ┆ 1          ┆ 2020-04-04  ┆ 4            ┆ 4              │
│ 2          ┆ 1          ┆ 2020-04-04  ┆ 4            ┆ 4              │
└────────────┴────────────┴─────────────┴──────────────┴────────────────┘


In [12]:
pl_lf2=(sample2.select(pl.col('student_id')
                       ,pl.col('date_birth')
                )
               .with_columns(birthday_day=pl.col('date_birth')
                                            .dt
                                            .day(),
                             birthday_month=pl.col('date_birth')
                                              .dt
                                              .month()
                )
).collect()
print(f'Identifying birthday only by month and day:\n{pl_lf2}')

Identifying birthday only by month and day:
shape: (4, 4)
┌────────────┬────────────┬──────────────┬────────────────┐
│ student_id ┆ date_birth ┆ birthday_day ┆ birthday_month │
│ ---        ┆ ---        ┆ ---          ┆ ---            │
│ i64        ┆ date       ┆ i8           ┆ i8             │
╞════════════╪════════════╪══════════════╪════════════════╡
│ 1          ┆ 2012-04-03 ┆ 3            ┆ 4              │
│ 2          ┆ 2013-04-04 ┆ 4            ┆ 4              │
│ 3          ┆ 2014-04-05 ┆ 5            ┆ 4              │
│ 4          ┆ 2013-04-03 ┆ 3            ┆ 4              │
└────────────┴────────────┴──────────────┴────────────────┘


In [13]:
pl_birthday=(pl_attendance.select(pl.col('student_id')
                                         ,pl.col('attendance')
                                         ,pl.col('school_date')
                           )
                          .with_columns(birthday_day=pl.col('school_date')
                                                       .dt
                                                       .day(),
                                        birthday_month=pl.col('school_date')
                                                         .dt
                                                         .month()
                           )
                          .join(pl_students.select(pl.col('student_id')
                                                   ,pl.col('date_birth')
                                            )
                                           .with_columns(birthday_day=pl.col('date_birth')
                                                                        .dt
                                                                        .day(),
                                                         birthday_month=pl.col('date_birth')
                                                                          .dt
                                                                          .month()
                                            ),
                                on=['student_id','birthday_day','birthday_month'],
                                how='right'
                           )
                          .select(pl.col('student_id')
                                  ,pl.col('attendance')
                           )
).collect()
fraction=(pl_birthday.select(pl.col('attendance')
                               .mean()
                               .round(2)
                      )
)
print(f'{pl_birthday.head(5)}\nfraction of attendances in birthday dates:\n{fraction}')

shape: (4, 2)
┌────────────┬────────────┐
│ student_id ┆ attendance │
│ ---        ┆ ---        │
│ i64        ┆ i64        │
╞════════════╪════════════╡
│ 1          ┆ 0          │
│ 2          ┆ 1          │
│ 3          ┆ 1          │
│ 4          ┆ null       │
└────────────┴────────────┘
fraction of attendances in birthday dates:
shape: (1, 1)
┌────────────┐
│ attendance │
│ ---        │
│ f64        │
╞════════════╡
│ 0.67       │
└────────────┘
